In [ ]:
from enum import Enum
from PIL import Image
from typing import Optional, Tuple
import numpy as np
import numpy.typing as npt
import matplotlib.pyplot as plt

In [ ]:
class FailReason(Enum):
    DIMENSIONS, GREYSCALE = range(2)

def correct_dimensions(im: npt.NDArray) -> bool:
    correct_width = np.round(im.shape[0] / 45 * 35)

    return im.shape[1] == correct_width

def is_color_image(im: npt.NDArray) -> bool:
    try:
        is_rgb = im.shape[2] == 3
    except IndexError:
        is_rgb == False

    return is_rgb and not (im[..., 0] == im[..., 1]).all() and not (im[..., 1] == im[..., 2]).all()

def is_passbild(im: npt.NDArray) -> Tuple[bool, Optional[FailReason]]:
    checks = {
        correct_dimensions: FailReason.DIMENSIONS,
        is_color_image: FailReason.GREYSCALE,
    }

    for check_func, fail_reason in checks.items():
        if not check_func(im):
            return False, fail_reason
    
    return True, None

In [ ]:
im = np.array(Image.open('inputs/kein_passbild.jpeg'))

reason_message = {
    FailReason.DIMENSIONS: 'Ein richtiges Passbild muss 45mm hoch und 35mm breit sein.',
    FailReason.GREYSCALE: 'Ein richtiges Passbild muss in Farbe sein.',
}

result, reason = is_passbild(im)

if result:
    title = 'Passbild :)'
    message = 'Das ist ein Passbild!'
else:
    title = 'Kein Passbild :('
    message = f'Das ist leider kein Passbild! {reason_message[reason]}'

plt.imshow(im)
plt.axis('off')
plt.title(title)
plt.show()

print(message)